In [ ]:
import pandas as pd
import os
import seaborn as sns
from statsmodels.regression.linear_model import OLS, WLS
import statsmodels.regression.linear_model as lm
import statsmodels.stats.diagnostic as smsdia
import statsmodels.stats.outliers_influence as oi
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
from plotly import __version__
print(__version__) # it should be above 1.9
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode,plot,iplot
# connected = True, plotly connects pandas to interactive java library
import colorlover as cl
init_notebook_mode(connected=True)
print(cf.__version__)
cf.go_offline()

%matplotlib inline

In [ ]:
bp = pd.read_csv('/Users/hari/Data/Building_Permits_Issued.csv')

In [ ]:
bp.head()

In [ ]:
building = bp.loc[bp['Permit Type Description'].str.contains('Building')]

In [ ]:
new_building = building.loc[building['Permit Type Description'].str.contains('New')]

In [ ]:
demolition = bp.loc[bp['Permit Type Description'].str.contains('Demolition')]

In [ ]:
res_demolition = demolition.loc[demolition['Permit Subtype Description'].str.contains('Residential')]

In [ ]:
merged = pd.merge(new_building, res_demolition, how='outer')

In [ ]:
#bp.loc[bp['Permit Subtype'].isin(['CAZ01A001', 'CAZ01A002'])]

In [ ]:
merged_sf = merged.loc[merged['Permit Subtype Description'].str.contains('Single Family Residence')]

In [ ]:
merged_sf.head()

In [ ]:
new_commercial = ['CAA03R299', 'CAA03R301', 'CAA03R398', 'CAA03R399', 'CAA05R202']

In [ ]:
new_residential = ['CAA01R301', 'CAA02R302', 'CAA03R298', 'CAA03R299', 'CAA03R301', 'CAA03R398', 'CAA03R399', 'CAA04R301', 'CAA05R202', 'CAA08R301', 'CAA10R301', 'CAZ08A001', 'DPLX2CNDO']

In [ ]:
matched = pd.merge(bp.loc[bp['Permit Subtype'].isin(['CAZ01A001', 'CAZ01A002'])], new_building.loc[new_building['Permit Subtype'].isin(new_commercial + new_residential)], on = 'Address')


In [ ]:
#bp['Permit Type Description'].value_counts()

In [ ]:
#bp['Permit Subtype Description'].value_counts()

In [ ]:
commercialdemo = bp.loc[bp['Permit Subtype']== 'CAZ01A001']

In [ ]:
sf_2009 = pd.read_csv('/Users/hari/Data/2009 single family_final_with_sqft.txt', dtype={'AddressPostalCode': 'str'})
sf_2013 = pd.read_csv('/Users/hari/Data/2013 single family_final_with_sqft.txt',dtype={'AddressPostalCode': 'str'})
sf_2017 = pd.read_csv('/Users/hari/Data/2017 single family_final_with_sqft.txt',dtype={'AddressPostalCode': 'str'})


In [ ]:
sf_2017.loc[sf_2017.AddressFullAddress== '605 SUMMIT OAKS CT']

#building = bp.loc[bp['Permit Type Description'].str.contains('Building')]

In [ ]:
print(sf_2009.shape)
print(sf_2009.columns)
print(sf_2009.duplicated().sum())
#sf_2009.loc[sf_2009.duplicated(),:]

In [ ]:
print(sf_2013.shape)
print(sf_2013.columns)
print(sf_2013.duplicated().sum())


In [ ]:
print(sf_2017.shape)
print(sf_2017.duplicated().sum())
sf_2017.sort_values(ascending =False, by ='2017 TOTAL APPR').describe()

In [ ]:
sf_2017.isnull().sum()

In [ ]:
sf1 = pd.merge(sf_2009, sf_2013, on= 'AddressFullAddress', how = 'outer',suffixes = ('_f','_s'))
sf1.columns

sf1.rename(columns = {'FinishedArea_f' : '2009_FinishedArea', 'FinishedArea_s' : '2013_FinishedArea'}, inplace = True)

sf1['APN']= sf1['APN_f'].where(sf1['APN_s'].isnull(), sf1['APN_s'])
sf1['DistrictCode']= sf1['DistrictCode_f'].where(sf1['DistrictCode_s'].isnull(), sf1['DistrictCode_s'])
sf1['Council District']= sf1['Council District_f'].where(sf1['Council District_s'].isnull(), sf1['Council District_s'])
sf1['AddressCity']= sf1['AddressCity_f'].where(sf1['AddressCity_s'].isnull(), sf1['AddressCity_s'])
sf1['AddressPostalCode']= sf1['AddressPostalCode_f'].where(sf1['AddressPostalCode_s'].isnull(), sf1['AddressPostalCode_s'])
#sf1['FinishedArea']= sf1['FinishedArea_f'].where(sf1['FinishedArea_s'].isnull(), sf1['FinishedArea_s'])
sf1.columns

sf1.drop(['APN_f','DistrictCode_f','Council District_f','AddressCity_f',
          'AddressPostalCode_f','DistrictCode_s','Council District_s','AddressCity_s','APN_s','AddressPostalCode_s'], axis = 1,inplace = True)

final_sf = pd.merge(sf1, sf_2017,on= 'AddressFullAddress', how = 'outer', suffixes = ('_m','_n'),indicator = True)
final_sf.columns

final_sf.rename(columns = {'FinishedArea': '2017_FinishedArea'}, inplace = True)

final_sf['APN']= final_sf['APN_m'].where(final_sf['APN_n'].isnull(), final_sf['APN_n'])
final_sf['DistrictCode']= final_sf['DistrictCode_m'].where(final_sf['DistrictCode_n'].isnull(), final_sf['DistrictCode_n'])
final_sf['Council District']= final_sf['Council District_m'].where(final_sf['Council District_n'].isnull(), final_sf['Council District_n'])
final_sf['AddressCity']= final_sf['AddressCity_m'].where(final_sf['AddressCity_n'].isnull(), final_sf['AddressCity_n'])
final_sf['AddressPostalCode']= final_sf['AddressPostalCode_m'].where(final_sf['AddressPostalCode_n'].isnull(), final_sf['AddressPostalCode_n'])
#final_sf['FinishedArea']= final_sf['FinishedArea_m'].where(final_sf['FinishedArea_n'].isnull(), final_sf['FinishedArea_n'])


final_sf.drop(['APN_m','DistrictCode_m','Council District_m','AddressCity_m',
          'AddressPostalCode_m','DistrictCode_n','Council District_n','AddressCity_n','APN_n','AddressPostalCode_n'], axis = 1, inplace = True)
final_sf.columns

final_sf.drop_duplicates(keep ='first', inplace = True)



In [ ]:
final_sf.head()

In [ ]:
final_sf.isnull().sum(axis = 0)
# count_nan = len(final_sf) - final_sf.count()
#final_sf.columns

In [ ]:
final_sf['Pct_Change_2009_2013'] = (final_sf['2013 TOTAL APPR'] - final_sf['2009 TOTAL APPR']) / final_sf['2009 TOTAL APPR']*100
final_sf['Pct_Change_2013_2017'] = (final_sf['2017 TOTAL APPR'] - final_sf['2013 TOTAL APPR']) / final_sf['2013 TOTAL APPR']*100
final_sf['Pct_Change_2009_2017'] = (final_sf['2017 TOTAL APPR'] - final_sf['2009 TOTAL APPR']) / final_sf['2009 TOTAL APPR']*100

In [ ]:
final_sf.shape

In [ ]:
r = 0.04 #Annual interest rate
y = 30 #Length of mortgage (years)
n = 12*y # Number of monthly payments
d = 0.05 # Down payment percentage
final_sf['Mortgage_Per_Year_2009'] = 12* (1-d)*final_sf['2009 TOTAL APPR'] * r/12 *(1+r/12)**n / ((1+r/12)**n -1)
final_sf['Mortgage_Per_Year_2013'] = 12*(1-d)*final_sf['2013 TOTAL APPR'] * r/12 *(1+r/12)**n / ((1+r/12)**n -1)
final_sf['Mortgage_Per_Year_2017'] = 12*(1-d)*final_sf['2017 TOTAL APPR'] * r/12 *(1+r/12)**n / ((1+r/12)**n -1)

In [ ]:
final_sf.head()

In [ ]:
#final_sf.groupby('Council District').Pct_Change_2013_2017.mean().nlargest(5)

In [ ]:
final_sf.groupby('AddressPostalCode').Pct_Change_2013_2017.mean().nlargest(5)

In [ ]:
final_sf.groupby('Council District').Pct_Change_2009_2017.mean().nlargest(5)

In [ ]:
# number of homes fell in 30% Affordable housing in 2009 based on 6% increase in AMI from 2009 t0 2017
final_sf.loc[final_sf['Mortgage_Per_Year_2009'] < 64900*.6*.3].AddressFullAddress.count()
#aff30.sort_values(ascending = False, by='2009 TOTAL APPR' ).columns

In [ ]:
# number of homes fell in 30% Affordable housing in 2013
final_sf.loc[final_sf['Mortgage_Per_Year_2013'] < 62300*.6*.3].AddressFullAddress.count()

In [ ]:
# number of homes fell in 30% Affordable housing in 2013
final_sf.loc[final_sf['Mortgage_Per_Year_2017'] < 68000*.6*.3].AddressFullAddress.count()

In [ ]:
aff30= final_sf[(final_sf['Mortgage_Per_Year_2009'] < 64900*.6*.3) 
                 & (final_sf['Mortgage_Per_Year_2013'] < 62300*.6*.3) 
                 &(final_sf['Mortgage_Per_Year_2017'] < 68000*.6*.3)]

In [ ]:
aff30[['2009_FinishedArea','2013_FinishedArea','2017_FinishedArea']].count()

In [ ]:
# print(aff30.groupby('Council District').Pct_Change_2009_2013.mean().nlargest(10))
# print(aff30.groupby('AddressCity').Pct_Change_2009_2013.mean().nlargest(10))


In [ ]:
aff30.loc[aff30['AddressPostalCode']=='37228']

In [ ]:
aff_cds = aff30.groupby('Council District',as_index=False)['2009 TOTAL APPR', '2013 TOTAL APPR','2017 TOTAL APPR','Pct_Change_2009_2013','Pct_Change_2013_2017','Pct_Change_2009_2017','2017_FinishedArea'].mean().round(decimals = 0).apply(np.int64)
aff_cds['Council District'] = pd.to_numeric(aff_cds['Council District'])
aff_cds.sort_values(ascending = False, by = 'Pct_Change_2009_2017')
aff_cds.head()

In [ ]:
aff_cds.plot.hexbin(x='2017 TOTAL APPR',y='2017_FinishedArea', figsize =(12,12),gridsize = 25, cmap ='Pastel1')

In [ ]:
# aff_cds_count = aff30.groupby('Council District',as_index=False)['2009 TOTAL APPR','2013 TOTAL APPR','2017 TOTAL APPR'].agg('count')
# aff_cds_count.count
# #.set_index('Council District').sort_values(ascending = False, by='Council District')

In [ ]:
# aff30[['Council District', '2009 TOTAL APPR', '2013 TOTAL APPR',
#        '2017 TOTAL APPR','Pct_Change_2009_2013','Pct_Change_2013_2017','Pct_Change_2009_2017']].groupby(['Council District']).agg(['mean', 'count'])

In [ ]:
aff_cds.describe()

In [ ]:
aff_cds.iplot(kind = 'scatter',x = 'Council District', y = ['Pct_Change_2009_2013','Pct_Change_2013_2017','Pct_Change_2009_2017'],
              xTitle='Council District',yTitle='Pct_Change_2009_2013',title='Percentage Change in affordable Housing in Each council district', ticks='outside')


In [ ]:
aff_cds.iplot(kind='box', x = 'Council District', y = ['2009 TOTAL APPR','2013 TOTAL APPR','2017 TOTAL APPR'])

In [ ]:
aff_cds.sort_values(ascending = False, by = 'Pct_Change_2009_2013').nlargest(10,"Pct_Change_2009_2017")

In [ ]:

# color = ['blue','yellow','green','purple', 'red','violet','pink', 'black',
#          'lightgrey','orange', 'blue','yellow','Cyan','green','purple', 
#          'violet','pink', 'black','lightgrey','orange','red','violet','pink', 
#          'black','lightgrey','orange', 'blue','orange','red','violet','pink', 
#          'black','lightgrey','orange', 'blue']

# aff_cds.iplot(kind = 'bubble', x='Council District',y='Pct_Change_2009_2013'
#               ,size = 'Pct_Change_2009_2017', xTitle='Council District',yTitle='Pct_Change_2009_2013',
#               title='Percentage Change in affordable Housing from 2009 - 2013 in each council district', 
#               ticks='outside', color = color, text = ['2009 TOTAL APPR','2013 TOTAL APPR','Council District'])
# aff_cds.iplot(kind = 'bubble', x='Council District',y='Pct_Change_2013_2017'
#               ,size = 'Pct_Change_2009_2017', xTitle='Council District',yTitle='Pct_Change_2013_2017',
#               title='Percentage Change in affordable Housing from 2013 - 2017 in each council district', 
#               ticks='outside',color = color,text = ['2013 TOTAL APPR','2017 TOTAL APPR','Council District'])
# aff_cds.iplot(kind = 'bubble', x='Council District',y='Pct_Change_2009_2017'
#               ,size = 'Pct_Change_2009_2017', xTitle='Council District',yTitle='Pct_Change_2009_2017',
#               title= 'Percentage Change in affordable Housing from 2009 - 2017 in each council district', 
#               ticks='outside',color = color,text = ['2009 TOTAL APPR','2017 TOTAL APPR','Council District'])

In [ ]:
affcity = aff30.groupby('AddressCity',as_index=False)['2009 TOTAL APPR', '2013 TOTAL APPR','2017 TOTAL APPR','Pct_Change_2009_2013','Pct_Change_2013_2017','Pct_Change_2009_2017'].mean().round(decimals = 0)
#aff2009['AddressPostalCode'] = pd.to_numeric(aff2009['AddressPostalCode'])
affcity.set_index('AddressCity')

In [ ]:
# color = ['blue','yellow','green','purple', 'red','violet','pink', 'black',
#          'lightgrey','orange', 'lightgrey','orange','violet','grey']
# affcity.iplot(kind = 'bubble', x='AddressCity',y='Pct_Change_2009_2013',size = 'Pct_Change_2009_2013',
#              xTitle='',yTitle='Pct_Change_2009_2013',color=color,
#                   title='Percentage Change in affordable Housing 2009-2013 by City', 
#               ticks='outside',text = ['2009 TOTAL APPR','2013 TOTAL APPR','AddressCity'])
# affcity.iplot(kind = 'bubble', x='AddressCity',y='Pct_Change_2013_2017',size = 'Pct_Change_2013_2017',
#              xTitle='',yTitle='Pct_Change_2013_2017',color=color,
#                   title='Percentage Change in affordable Housing 2013-2017 by City', ticks='outside',
#               text = ['2013 TOTAL APPR','2017 TOTAL APPR','AddressCity'])
# affcity.iplot(kind = 'bubble', x='AddressCity',y='Pct_Change_2009_2017',size = 'Pct_Change_2009_2017',
#              xTitle='',yTitle='Pct_Change_2009_2017',color=color,
#                   title='Percentage Change in affordable Housing 2009-2017  by City', ticks='outside',
#                           text = ['2013 TOTAL APPR','2017 TOTAL APPR','AddressCity'])


In [ ]:
workforce_aff= final_sf[(final_sf['Mortgage_Per_Year_2009'] < 64900*1.2*.3) & (final_sf['Mortgage_Per_Year_2009'] > 64900*.6*.3)
                       & (final_sf['Mortgage_Per_Year_2013'] < 62300*1.2*.3) & (final_sf['Mortgage_Per_Year_2013'] > 62300*.6*.3)
                       & (final_sf['Mortgage_Per_Year_2017'] < 68000*1.2*.3) & (final_sf['Mortgage_Per_Year_2017'] > 68000*.6*.3)]

In [ ]:
print(workforce_aff.shape)

In [ ]:
print(aff30.shape)

In [ ]:
# tax_2016 = pd.read_csv('/Users/hari/Data/2016tax.csv')
# tax_2016.head()

In [ ]:
#tax_2016.loc[tax_2016.STATE=='TN']

In [ ]:
print(68000*1.2*.3)

In [ ]:
workforce_cds = workforce_aff.groupby('Council District',as_index=False)['2009 TOTAL APPR', '2013 TOTAL APPR','2017 TOTAL APPR','Pct_Change_2009_2013','Pct_Change_2013_2017','Pct_Change_2009_2017'].mean().round(decimals = 0).apply(np.int64)
workforce_cds['Council District'] = pd.to_numeric(workforce_cds['Council District'])
workforce_cds.sort_values(ascending = False, by = 'Pct_Change_2009_2017')
workforce_cds.head()

In [ ]:
workforce_cds.describe()

In [ ]:
aff_drop = aff_cds.drop(['Council District','Pct_Change_2009_2013','Pct_Change_2013_2017','Pct_Change_2009_2017','2017_FinishedArea'], axis = 1)
work_drop =workforce_cds.drop(['Council District','Pct_Change_2009_2013','Pct_Change_2013_2017','Pct_Change_2009_2017'], axis = 1)
aff_drop.head()

In [ ]:
# color = ['blue','yellow','green','purple', 'red','violet','pink', 'black',
#          'lightgrey','orange', 'blue','yellow','Cyan','green','purple', 
#          'violet','pink', 'black','lightgrey','orange','red','violet','pink', 
#          'black','lightgrey','orange', 'blue','orange','red','violet','pink', 
#          'black','lightgrey','orange', 'blue']

# workforce_cds.iplot(kind = 'bubble', x='Council District',y='Pct_Change_2009_2013'
#               ,size = 'Pct_Change_2009_2017', xTitle='Council District',yTitle='Pct_Change_2009_2013',
#               title='Percentage Change in work force Housing  2009 - 2013 in each council district based on Appraisal', 
#               ticks='outside', color = color, text = ['2009 TOTAL APPR','2013 TOTAL APPR','Council District'])
# workforce_cds.iplot(kind = 'bubble', x='Council District',y='Pct_Change_2013_2017'
#               ,size = 'Pct_Change_2009_2017', xTitle='Council District',yTitle='Pct_Change_2013_2017',
#               title='Percentage Change in work force Housing from 2013 - 2017 in each council district based on Appraisal', 
#               ticks='outside',color = color,text = ['2013 TOTAL APPR','2017 TOTAL APPR','Council District'])
# workforce_cds.iplot(kind = 'bubble', x='Council District',y='Pct_Change_2009_2017'
#               ,size = 'Pct_Change_2009_2017', xTitle='Council District',yTitle='Pct_Change_2009_2017',
#               title= 'Percentage Change in work force Housing from 2009 - 2017 in each council district based on Appraisal', 
#               ticks='outside',color = color,text = ['2009 TOTAL APPR','2017 TOTAL APPR','Council District'])

In [ ]:
workforcecity = workforce_aff.groupby('AddressCity',as_index=False)['2009 TOTAL APPR', '2013 TOTAL APPR','2017 TOTAL APPR','Pct_Change_2009_2013','Pct_Change_2013_2017','Pct_Change_2009_2017'].mean().round(decimals = 0)
#aff2009['AddressPostalCode'] = pd.to_numeric(aff2009['AddressPostalCode'])
affcity.set_index('AddressCity')

In [ ]:
#workforcecity.sort_values(ascending = False, by = 'Pct_Change_2009_2017')

In [ ]:
# color = ['blue','yellow','green','purple', 'red','violet','pink', 'black',
#          'lightgrey','orange', 'lightgrey','orange','violet']
# workforcecity.iplot(kind = 'bubble', x='AddressCity',y='Pct_Change_2009_2013',size = 'Pct_Change_2009_2013',
#              xTitle='',yTitle='Pct_Change_2009_2013',color=color,
#                   title='Percentage Change in workforce Housing 2009-2013 by City', 
#               ticks='outside',text = ['2009 TOTAL APPR','2013 TOTAL APPR','AddressCity'])
# workforcecity.iplot(kind = 'bubble', x='AddressCity',y='Pct_Change_2013_2017',size = 'Pct_Change_2013_2017',
#              xTitle='',yTitle='Pct_Change_2013_2017',color=color,
#                   title='Percentage Change in workforce Housing 2013-2017 by City', ticks='outside',
#               text = ['2013 TOTAL APPR','2017 TOTAL APPR','AddressCity'])
# workforcecity.iplot(kind = 'bubble', x='AddressCity',y='Pct_Change_2009_2017',size = 'Pct_Change_2009_2017',
#              xTitle='',yTitle='Pct_Change_2009_2017',color=color,
#                title='Percentage Change in workforce Housing 2009-2017  by City', ticks='outside',
#                           text = ['2013 TOTAL APPR','2017 TOTAL APPR','AddressCity'])


In [ ]:
aff_geo = gpd.read_file('/Users/hari/Data/Council District Outlines (GIS).geojson')
aff_geo['district'] = pd.to_numeric(aff_geo['district'])

In [ ]:
aff_cds.columns

In [ ]:
merged = pd.merge(aff_geo,
                  aff_cds, left_on ='district', right_on ='Council District')

In [ ]:
merged.drop(['district'], axis = 1, inplace = True)

In [ ]:
merged.columns

In [ ]:
merged.sort_values(ascending = False, by ='Pct_Change_2009_2017').head()

In [ ]:
merged['coords'] = merged['geometry'].apply(lambda x: x.representative_point().coords[:])
merged['coords'] = [coords[0] for coords in merged['coords']]
merged.plot()
for idx, row in merged.iterrows():
    plt.annotate(s=row['Council District'], xy=row['coords'],
                 horizontalalignment='center')
# ax= merged.plot()
# merged.apply(lambda x: ax.annotate(s=x['Council District'], xy=x.geometry.centroid.coords[0], ha='center'),axis=1);

In [ ]:
leg_kwds = {'title': 'Council District', 'loc': 'upper left', 'bbox_to_anchor': (1, 1.03), 'ncol': 1}
merged.plot(column = '2009 TOTAL APPR', figsize=(20,20),cmap='OrRd', edgecolor = 'black',legend = True,legend_kwds = leg_kwds)
plt.title("Affordable housing 30% \n Pct_Change_2009_2013 by Council District")
for idx, row in merged.iterrows():
    plt.annotate(s=row['2009 TOTAL APPR'], xy=row['coords'],
                 horizontalalignment='center')


leg_kwds = {'title': 'Council District', 'loc': 'upper left', 'bbox_to_anchor': (1, 1.03), 'ncol': 1}
merged.plot(column = '2017 TOTAL APPR', figsize=(20,20), cmap='OrRd',edgecolor = 'black',legend = True,legend_kwds = leg_kwds)
plt.title("Affordable housing 30% \n Pct_Change_2009_2017 by Council District")
for idx, row in merged.iterrows():
    plt.annotate(s=row['2017 TOTAL APPR'], xy=row['coords'],
                 horizontalalignment='center')

## Affordable and Workforce Housing counts based on 2009, 2013, and 2017 AMI and mortgage income.


In [ ]:
final_sf.columns

## Per district housing count

In [ ]:
aff_2009 = final_sf.loc[final_sf.Mortgage_Per_Year_2009 < 64900*.6*.3].groupby('Council District').AddressFullAddress.count().to_frame().rename(columns = {'AddressFullAddress': 'count_2009'}).reset_index()

aff_2013 = final_sf.loc[final_sf.Mortgage_Per_Year_2013 < 62300*.6*.3].groupby('Council District').AddressFullAddress.count().to_frame().rename(columns = {'AddressFullAddress': 'count_2013'}).reset_index()

aff_2017 = final_sf.loc[final_sf.Mortgage_Per_Year_2017 < 68000*.6*.3].groupby('Council District').AddressFullAddress.count().to_frame().rename(columns = {'AddressFullAddress': 'count_2017'}).reset_index()

aff_per_district = pd.merge(pd.merge(aff_2009, aff_2013), aff_2017)

aff_per_district['Change_2009_2013'] = aff_per_district.count_2013 - aff_per_district.count_2009
aff_per_district['Change_2013_2017'] = aff_per_district.count_2017 - aff_per_district.count_2013
aff_per_district['Change_2009_2017'] = aff_per_district.count_2017 - aff_per_district.count_2009

aff_per_district['Pct_Change_2009_2013'] = (aff_per_district.count_2013 - aff_per_district.count_2009) / aff_per_district.count_2009*100
aff_per_district['Pct_Change_2013_2017'] = (aff_per_district.count_2017 - aff_per_district.count_2013) / aff_per_district.count_2013*100
aff_per_district['Pct_Change_2009_2017'] = (aff_per_district.count_2017 - aff_per_district.count_2009) / aff_per_district.count_2009*100

## Per city Affordable housing count

In [ ]:
aff_2009_city = final_sf.loc[final_sf.Mortgage_Per_Year_2009 < 64900*.6*.3].groupby('AddressCity').AddressFullAddress.count().to_frame().rename(columns = {'AddressFullAddress': 'count_2009'}).reset_index()

aff_2013_city = final_sf.loc[final_sf.Mortgage_Per_Year_2013 < 62300*.6*.3].groupby('AddressCity').AddressFullAddress.count().to_frame().rename(columns = {'AddressFullAddress': 'count_2013'}).reset_index()

aff_2017_city= final_sf.loc[final_sf.Mortgage_Per_Year_2017 < 68000*.6*.3].groupby('AddressCity').AddressFullAddress.count().to_frame().rename(columns = {'AddressFullAddress': 'count_2017'}).reset_index()

aff_per_city = pd.merge(pd.merge(aff_2009_city, aff_2013_city), aff_2017_city)

aff_per_city['Change_2009_2013'] = aff_per_city.count_2013 - aff_per_city.count_2009
aff_per_city['Change_2013_2017'] = aff_per_city.count_2017 - aff_per_city.count_2013
aff_per_city['Change_2009_2017'] = aff_per_city.count_2017 - aff_per_city.count_2009

aff_per_city['Pct_Change_2009_2013'] = (aff_per_city.count_2013 - aff_per_city.count_2009) / aff_per_city.count_2009*100
aff_per_city['Pct_Change_2013_2017'] = (aff_per_city.count_2017 - aff_per_city.count_2013) / aff_per_city.count_2013*100
aff_per_city['Pct_Change_2009_2017'] = (aff_per_city.count_2017 - aff_per_city.count_2009) / aff_per_city.count_2009*100

In [ ]:
df1 = affcity.rename(columns = {'Pct_Change_2009_2013':'Pct_Change_APPR_2009_2013','Pct_Change_2013_2017':'Pct_Change_APPR_2023_2017','Pct_Change_2009_2017':'Pct_Change_APPR_2009_2017'})
aff_city =pd.merge(aff_per_city, df1, left_on ='AddressCity', right_on ='AddressCity').round(decimals = 1)
aff_city.head()


## Per District workforce housing count

In [ ]:
workforce_2009 = final_sf.loc[(final_sf['Mortgage_Per_Year_2009'] < 64900*1.2*.3) & (final_sf['Mortgage_Per_Year_2009'] > 64900*.6*.3)] \
.groupby('Council District').AddressFullAddress.count().to_frame().rename(columns = {'AddressFullAddress': 'count_2009'}).reset_index()
workforce_2013 = final_sf.loc[(final_sf['Mortgage_Per_Year_2013'] < 62300*1.2*.3) & (final_sf['Mortgage_Per_Year_2013'] > 62300*.6*.3)] \
.groupby('Council District').AddressFullAddress.count().to_frame().rename(columns = {'AddressFullAddress': 'count_2013'}).reset_index()
workforce_2017= final_sf.loc[(final_sf['Mortgage_Per_Year_2017'] < 68000*1.2*.3) & (final_sf['Mortgage_Per_Year_2017'] > 68000*.6*.3)] \
.groupby('Council District').AddressFullAddress.count().to_frame().rename(columns = {'AddressFullAddress': 'count_2017'}).reset_index()

workforce_per_district = pd.merge(pd.merge(workforce_2009, workforce_2013), workforce_2017)
workforce_per_district['Change_2009_2013'] = workforce_per_district.count_2013 - workforce_per_district.count_2009
workforce_per_district['Change_2013_2017'] = workforce_per_district.count_2017 - workforce_per_district.count_2013
workforce_per_district['Change_2009_2017'] = workforce_per_district.count_2017 - workforce_per_district.count_2009


workforce_per_district['Pct_Change_2009_2013'] = (workforce_per_district.count_2013 - workforce_per_district.count_2009) / workforce_per_district.count_2009*100
workforce_per_district['Pct_Change_2013_2017'] = (workforce_per_district.count_2017 - workforce_per_district.count_2013) / workforce_per_district.count_2013*100
workforce_per_district['Pct_Change_2009_2017'] = (workforce_per_district.count_2017 - workforce_per_district.count_2009) / workforce_per_district.count_2009*100

## Per City workforce housing count

In [ ]:
workforce_2009_city = final_sf.loc[(final_sf['Mortgage_Per_Year_2009'] < 64900*1.2*.3) & (final_sf['Mortgage_Per_Year_2009'] > 64900*.6*.3)] \
.groupby('AddressCity').AddressFullAddress.count().to_frame().rename(columns = {'AddressFullAddress': 'count_2009'}).reset_index()
workforce_2013_city = final_sf.loc[(final_sf['Mortgage_Per_Year_2013'] < 62300*1.2*.3) & (final_sf['Mortgage_Per_Year_2013'] > 62300*.6*.3)] \
.groupby('AddressCity').AddressFullAddress.count().to_frame().rename(columns = {'AddressFullAddress': 'count_2013'}).reset_index()
workforce_2017_city= final_sf.loc[(final_sf['Mortgage_Per_Year_2017'] < 68000*1.2*.3) & (final_sf['Mortgage_Per_Year_2017'] > 68000*.6*.3)] \
.groupby('AddressCity').AddressFullAddress.count().to_frame().rename(columns = {'AddressFullAddress': 'count_2017'}).reset_index()

workforce_per_city = pd.merge(pd.merge(workforce_2009_city, workforce_2013_city), workforce_2017_city)
workforce_per_city['Change_2009_2013'] = workforce_per_city.count_2013 - workforce_per_city.count_2009
workforce_per_city['Change_2013_2017'] = workforce_per_city.count_2017 - workforce_per_city.count_2013
workforce_per_city['Change_2009_2017'] = workforce_per_city.count_2017 - workforce_per_city.count_2009


workforce_per_city['Pct_Change_2009_2013'] = (workforce_per_city.count_2013 - workforce_per_city.count_2009) / workforce_per_city.count_2009*100
workforce_per_city['Pct_Change_2013_2017'] = (workforce_per_city.count_2017 - workforce_per_city.count_2013) / workforce_per_city.count_2013*100
workforce_per_city['Pct_Change_2009_2017'] = (workforce_per_city.count_2017 - workforce_per_city.count_2009) / workforce_per_city.count_2009*100

In [ ]:
workforce_per_city

In [ ]:
df2 = workforcecity.rename(columns = {'Pct_Change_2009_2013':'Pct_Change_APPR_2009_2013','Pct_Change_2013_2017':'Pct_Change_APPR_2023_2017','Pct_Change_2009_2017':'Pct_Change_APPR_2009_2017'})
workforce_city =pd.merge(workforce_per_city, df2, left_on ='AddressCity', right_on ='AddressCity').round(decimals = 1)


In [ ]:
aff_city

In [ ]:
workforce_city

## Interactive plots for Affordable(30%AMI) and workforce(>30% and <120%) housing based on council district and House appraisal

In [ ]:
aff_drop.iplot(kind='box', boxpoints ='outliers', yTitle='Appraisal Value',
              title='Affordable Housing (30% of AMI) in each council district based on Appraisal', 
              ticks='outside', text = ['2009 TOTAL APPR','2013 TOTAL APPR','Council District'])
work_drop.iplot(kind='box',boxpoints ='outliers',yTitle='Appraisal Value',
                title='WorkForce Housing (30% of AMI) in each council district based on Appraisal', 
              ticks='outside', text = ['2009 TOTAL APPR','2017 TOTAL APPR','Council District'])

## Interactive Bubble plot for number of single family homes based on city

In [ ]:
color = ['blue','yellow','green','purple', 'red','Maroon','pink', 'black',
         'lightgrey','Cyan', 'Lime','orange','violet','indigo','tomato']
text = ['AddressCity','count_2009','count_2017','2009 TOTAL APPR','2017 TOTAL APPR','Pct_Change_APPR_2009_2017']

aff_city.iplot(kind = 'bubble',theme = 'white', x='AddressCity',y='Pct_Change_2009_2017',size = 'Pct_Change_2009_2017',
             xTitle='',yTitle='Percent Change from 2009 to 2017',color=color,
                  title='Affordable Housing (30% of AMI) Pct_Change_2009_2017 by City', ticks='outside',
                           text = text, hline=[1])


workforce_city.iplot(kind = 'bubble',theme = 'white', x='AddressCity',y='Pct_Change_2009_2017',size = 'Pct_Change_2009_2017',
             xTitle='',yTitle='Percent Change from 2009 to 2017',color=color,
                          title='WorkForce Housing (>30% and <120% AMI) Pct_Change_2009_2017 by Council District', ticks='outside',
                          text = text,hline=[1])
                                      

In [ ]:
aff_per_district.sort_values(ascending=False, by = 'Pct_Change_2009_2017').round(decimals = 1)

In [ ]:
workforce_per_district.sort_values(ascending=False, by = 'Pct_Change_2009_2017')

In [ ]:
df3 = gpd.read_file('/Users/hari/Data/Zip Codes (GIS)(.geojson')



In [ ]:
merged2 = pd.merge(df3,
                  aff_city, left_on ='po_name', right_on ='AddressCity')
merged3 = pd.merge(df3,
                  workforce_city, left_on ='po_name', right_on ='AddressCity')

In [ ]:
merged2['coords'] = merged2['geometry'].apply(lambda x: x.representative_point().coords[:])
merged2['coords'] = [coords[0] for coords in merged2['coords']]
merged2.plot()
for idx, row in merged2.iterrows():
    plt.annotate(s=row['AddressCity'], xy=row['coords'],
                 horizontalalignment='center')

In [ ]:
merged3['coords'] = merged3['geometry'].apply(lambda x: x.representative_point().coords[:])
merged3['coords'] = [coords[0] for coords in merged3['coords']]
merged3.plot()
for idx, row in merged3.iterrows():
    plt.annotate(s=row['AddressCity'], xy=row['coords'],
                 horizontalalignment='center')

In [ ]:
leg_kwds = {'title': 'City', 'loc': 'upper left', 'bbox_to_anchor': (1, 1.03), 'ncol': 1}
merged2.plot(column = 'Pct_Change_2009_2017', figsize=(25,25), cmap='OrRd',edgecolor = 'black',legend = True,legend_kwds = leg_kwds)
plt.title("Affordable housing 30% \n Pct_Change_2009_2017 by Council District")
for idx, row in merged2.iterrows():
    plt.annotate(s=row['AddressCity'], xy=row['coords'],
                 horizontalalignment='center', fontsize = 10)


leg_kwds = {'title': 'City', 'loc': 'upper left', 'bbox_to_anchor': (1, 1.03), 'ncol': 1}
merged3.plot(column = 'Pct_Change_2009_2017', figsize=(25,25), cmap='OrRd',edgecolor = 'black',legend = True,legend_kwds = leg_kwds)
plt.title("Workforce housing 30% \n Pct_Change_2009_2017 by Council District")
for idx, row in merged3.iterrows():
    plt.annotate(s=row['AddressCity'], xy=row['coords'],
                 horizontalalignment='center', fontsize = 10)